# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [33]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint

# Import API key
from api_keys import g_key
gmaps.configure(api_key = g_key)

In [35]:
#load csv into dataframe
load_csv = "cities.csv"
city_df = pd.read_csv(load_csv)
city_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


In [36]:
#humidity heatmap

locations = city_df[["Lat", "Lng"]]
humidity = city_df['Humidity'].astype(int)


# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [37]:

# Create new DataFrame fitting weather criteria
# max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.

narrowed_city_df = city_df.copy()
narrowed_city_df = narrowed_city_df[narrowed_city_df['Max Temp'] >= 70]
narrowed_city_df = narrowed_city_df[narrowed_city_df['Max Temp'] <= 80]  
narrowed_city_df = narrowed_city_df[narrowed_city_df['Wind Speed'] <= 10]
narrowed_city_df = narrowed_city_df[narrowed_city_df['Cloudiness'] == 0]
narrowed_city_df = narrowed_city_df.dropna()
narrowed_city_df = narrowed_city_df.reset_index()
narrowed_city_df

,index,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
1,88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
2,138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
3,139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
4,176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93
5,297,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77
6,444,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17


In [21]:
narrowed_city_df = cities.loc[(cities["Max Temp"] > 70) & (cities["Max Temp"] < 80) & (cities["Cloudiness"] == 0), :]
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df.reset_index(inplace=True)
del narrowed_city_df['index']
narrowed_city_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,56,pisco,0,PE,1558378764,30,-13.71,-76.20,75.99,11.41
1,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
2,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
3,114,koumac,0,NC,1558378775,68,-20.56,164.28,74.40,12.86
4,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06


In [38]:
# create hotel_df with hotel name column
hotel_df = narrowed_city_df
hotel_df["Hotel Name"] = np.nan
hotel_df = hotel_df.drop(columns='index')
hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32,NaN
1,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50,NaN
2,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06,NaN
3,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59,NaN
4,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93,NaN
5,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77,NaN
6,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17,NaN


In [39]:

#search googleplaces for hotels within 5000 meters
#list for hotel names
hotels = []
#checker variable
x = 0

# for loop to search through city coords
for cities in hotel_df['City']:
    target_coordinates = str(hotel_df['Lat'][x])+' , '+str(hotel_df['Lng'][x])
    x+=1
    target_radius = 5000
    target_type = 'lodging'
    params = {
    "location": target_coordinates,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
    }
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# try/except to check if a city turns up    
    try:
        response = requests.get(base_url, params=params)
        hotel_results = response.json()
        hotels.append(hotel_results['results'][0]['name'])
        
    except IndexError:
        hotels.append(np.nan)

#append dataframe with hotel names
hotel_df['Hotel Name'] = hotels
hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32,Hotel Escarpas do Lago
1,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50,Pond Country Club
2,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06,La Case Bambou
3,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59,Dandeli Lake County
4,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93,Hôtel La Giralda
5,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77,MOGOK HILL HOTEL
6,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17,تاکسی سرویس آرامش


In [40]:
#clean up DataSet by remooving NAN
hotel_df_clean = hotel_df.dropna()
hotel_df_clean

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32,Hotel Escarpas do Lago
1,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50,Pond Country Club
2,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06,La Case Bambou
3,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59,Dandeli Lake County
4,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93,Hôtel La Giralda
5,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77,MOGOK HILL HOTEL
6,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17,تاکسی سرویس آرامش


In [41]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df_clean.iterrows()]
locations = hotel_df_clean[["Lat", "Lng"]]

In [42]:
# Add marker layer ontop of heat map and display
fig = gmaps.figure()
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))